# Imports

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

from task_three import task_three
from utils import general_utils, data_prep_utils, feature_selection_utils


In [2]:
tcga_genes = data_prep_utils.gene_exp_log_trans(data_prep_utils.get_df("tcga_rna", nb=True))
tcga_muts = data_prep_utils.get_df("tcga_mut", nb=True)
tcga_muts_int = data_prep_utils.mut_df_label_to_int(tcga_muts)

In [3]:
tcga_muts_int.head()

,DNMT3A,IDH1,NPM1,IDH2,RUNX1,NRAS,CEBPA,FLT3,TP53,TET2,WT1
TCGA.AB.2803,0,0,0,0,0,0,0,0,0,0,0
TCGA.AB.2805,0,0,0,1,1,0,0,0,0,0,0
TCGA.AB.2806,0,0,0,0,0,0,0,0,0,0,0
TCGA.AB.2807,0,0,0,1,1,0,0,0,0,0,0
TCGA.AB.2808,0,0,0,0,0,1,1,0,0,0,0


# Test build_cor_mat

In [5]:
drugs_dict = {"d1": [0, 1, 2], "d2": [3, 4, 5]}
mut_dict = {"mut1": [1, 2, 3], "mut2": [3, 4, 5], "mut3": [18, 2, 97]}

drugs_df = pd.DataFrame.from_dict(drugs_dict)
print(drugs_df)
mut_df = pd.DataFrame.from_dict(mut_dict)
print(mut_df)

cor_mat = task_three.build_cor_mat(drugs_df, mut_df)
print(cor_mat)
testings_utils.export_drugs_prediction(cor_mat, "test_cor_mat")

   d1  d2
0   0   3
1   1   4
2   2   5
   mut1  mut2  mut3
0     1     3    18
1     2     4     2
2     3     5    97
            d1        d2
mut1  1.000000  1.000000
mut2  1.000000  1.000000
mut3  0.776603  0.776603


# Test get_linear_svc_model

In [3]:
x_train, x_test, y_train, y_test = train_test_split(tcga_genes, tcga_muts, test_size = 0.2, random_state = 42)

In [7]:
svc_model = task_three.get_linear_svc_model(tcga_genes, tcga_muts)

C:\Users\yaelz\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:1199: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


KeyboardInterrupt: 

# Test get_gradient_boosting_class_model

In [6]:
gbc = task_three.get_gradient_boosting_class_model(x_train, y_train)

KeyboardInterrupt: 

# Test KNN neighbors

### feature selections

In [4]:
tcga_muts_int.iloc[:, [0,2]]

,DNMT3A,NPM1
TCGA.AB.2803,0,0
TCGA.AB.2805,0,0
TCGA.AB.2806,0,0
TCGA.AB.2807,0,0
TCGA.AB.2808,0,0
...,...,...
TCGA.AB.3007,0,0
TCGA.AB.3008,0,0
TCGA.AB.3009,0,0
TCGA.AB.3011,0,1


In [5]:
tcga_genes_filtered, filtered_genes = feature_selection_utils.select_features_by_corrlation(tcga_genes, tcga_muts_int)

1  -  DNMT3A  ->  19794
2  -  IDH1  ->  19687
3  -  NPM1  ->  18744
4  -  IDH2  ->  18704
5  -  RUNX1  ->  18285
6  -  NRAS  ->  18273
7  -  CEBPA  ->  18161
8  -  FLT3  ->  18030
9  -  TP53  ->  16835
10  -  TET2  ->  16832
11  -  WT1  ->  16808


,A2ML1,AAGAB,AASS,ABCA11P,ABCA12,ABCA6,ABCA8,ABCA9,ABCB1,ABCB11,...,ZNF93,ZNF98,ZNF99,ZNRF2,ZNRF3,ZP4,ZPBP2,ZSCAN1,ZSWIM2,ZSWIM7
TCGA.AB.2803,3.337524,5.171199,3.498801,3.919211,3.321928,3.352954,3.321928,3.442219,3.491848,3.321928,...,4.758340,3.360607,3.352954,5.122514,3.329747,3.321928,3.321928,3.321928,3.321928,5.250306
TCGA.AB.2805,3.351531,5.804736,3.380538,3.807680,3.436860,4.012245,3.327897,4.336258,3.924361,3.321928,...,4.712136,3.408974,3.321928,6.626003,3.908515,3.321928,3.327897,3.321928,3.321928,5.038901
TCGA.AB.2806,3.369137,5.457113,3.399773,4.284078,3.394712,3.369137,3.327250,3.337837,4.226707,3.327250,...,5.807749,3.562064,3.321928,5.328994,4.357317,3.321928,3.321928,3.321928,3.321928,5.021326
TCGA.AB.2807,3.329710,5.359172,6.337252,3.575585,3.604485,3.940602,4.099254,3.813169,6.049864,3.345149,...,4.456364,3.654915,3.321928,7.043438,3.883589,3.321928,3.329710,3.352808,3.321928,4.413873
TCGA.AB.2808,3.370487,5.662129,3.443637,4.107884,3.342939,3.370487,3.377293,3.390808,5.130835,3.321928,...,4.786837,3.377293,3.321928,6.461005,3.637504,3.321928,3.356779,3.321928,3.321928,4.918639


In [13]:
for a,b in enumerate(tcga_muts_int.columns):
    print(a, b)

0 DNMT3A
1 IDH1
2 NPM1
3 IDH2
4 RUNX1
5 NRAS
6 CEBPA
7 FLT3
8 TP53
9 TET2
10 WT1
